##Using BERT

In [ ]:
import pandas as pd

path="hamvsspam.csv"

df = pd.read_csv("hamvsspam.csv", encoding='latin1')

# Keep only the first two columns (label and message)
df = df.iloc[:, :2]

# Rename the columns
df.columns = ['label', 'message']

print(df.head())



  label                                            message
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [ ]:

X=df['message']
y=df['label']

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [ ]:
X_train_l = X_train.tolist()
X_test_l = X_test.tolist()


In [ ]:
!pip install hf_xet
import numpy as np
from torch.utils.data import DataLoader, TensorDataset

# Set up model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def get_bert_embeddings_batched(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        encodings = tokenizer(batch_texts, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**encodings)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            embeddings.append(cls_embeddings.cpu().numpy())
    return np.concatenate(embeddings, axis=0)

# Assume `train_texts` and `test_texts` are lists of strings
X_train_embb = get_bert_embeddings_batched(X_train_l)
X_test_embb = get_bert_embeddings_batched(X_test_l)

# Train SVM
svm = SVC(kernel='linear')
svm.fit(X_train_embb, y_train)

y_pred_train = svm.predict(X_train_embb)
print(classification_report(y_train, y_pred_train))

y_pred_test = svm.predict(X_test_embb)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00      4987
        spam       1.00      1.00      1.00       767

    accuracy                           1.00      5754
   macro avg       1.00      1.00      1.00      5754
weighted avg       1.00      1.00      1.00      5754

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1645
        spam       0.98      0.96      0.97       274

    accuracy                           0.99      1919
   macro avg       0.99      0.98      0.98      1919
weighted avg       0.99      0.99      0.99      1919

